In [1]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/3c/e3/e868f1d5951047f950d2ba1e04a765a3328a51f06996b67976d6102f8227/tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/f6/d4/349f7f4bd5ea92dab34f5bb0fe31775ef6c311427a14d5a5b31ecb442341/absl_py-2.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/b8/25/155f9f080d5e4bc0082edfda032ea2bc2b8fab3f4d25d46c1e9dd22a1a89/flatbuffers-25.2.10-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.19.0
[]


In [3]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
  Obtaining dependency information for torch from https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for filelock from https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata
  Obtaining dependency information for jinja2 from https://download.pytorch.org/whl/Jinja2-3.1.4-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec from https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-a


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
print(torch.cuda.get_device_name(0))

NVIDIA RTX A4000


In [8]:
path='./CICIoT2023/'

In [10]:
!pip install "xgboost[dask]"  pandas numpy tqdm scikit-learn



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from collections import Counter

csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]

def feature_selection_summary(df, label_col='label', var_threshold=0.01, corr_threshold=0.95):
    X = df.drop(columns=[label_col])
    y = df[label_col]

    # Step 1: Low Variance Filter
    var_thresh = VarianceThreshold(threshold=var_threshold)
    X_low_var = var_thresh.fit_transform(X)
    low_var_selected_features = X.columns[var_thresh.get_support()].tolist()

    # Step 2: Pearson Correlation Filter
    X_low_var_df = pd.DataFrame(X_low_var, columns=low_var_selected_features)
    corr_matrix = X_low_var_df.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    high_corr_features = [col for col in upper_triangle.columns if any(upper_triangle[col] > corr_threshold)]
    X_selected = X_low_var_df.drop(columns=high_corr_features)

    final_selected_features = X_selected.columns.tolist()

    return {
        "Original feature count": X.shape[1],
        "After low variance filter": len(low_var_selected_features),
        "Highly correlated features dropped": high_corr_features,
        "Final selected features": final_selected_features,
        "Final feature count": len(final_selected_features)
    }

all_final_features = []

for file in csv_files:
    file_path = os.path.join(path, file)
    try:
        print(f"Processing file: {file}")
        df = pd.read_csv(file_path)

        if df.shape[0] < 10 or 'label' not in df.columns:
            print("Skipping: missing label or too few rows.")
            continue

        summary = feature_selection_summary(df)
        all_final_features.append(set(summary["Final selected features"]))

        # Print summary
        print(f"Original features: {summary['Original feature count']}")
        print(f"Remaining after variance filter: {summary['After low variance filter']}")
        print(f"Removed highly correlated: {len(summary['Highly correlated features dropped'])}")
        print(f"Final selected features: {summary['Final feature count']}")

    except Exception as e:
        print(f"Error processing {file}: {e}")

feature_counter = Counter()
for features in all_final_features:
    feature_counter.update(features)

N = len(csv_files) // 2 
final_voted_features = [feature for feature, count in feature_counter.items() if count >= N]

print(f"\nTotal number of files processed: {len(all_final_features)}")
print(f"Number of final voted features (appeared in >= {N} files): {len(final_voted_features)}")
print("Final voted feature names:")
print(final_voted_features)


Processing file: part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Original features: 46
Remaining after variance filter: 34
Removed highly correlated: 8
Final selected features: 26
Processing file: part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Original features: 46
Remaining after variance filter: 34
Removed highly correlated: 8
Final selected features: 26
Processing file: part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Original features: 46
Remaining after variance filter: 34
Removed highly correlated: 8
Final selected features: 26
Processing file: part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Original features: 46
Remaining after variance filter: 34
Removed highly correlated: 8
Final selected features: 26
Processing file: part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
Original features: 46
Remaining after variance filter: 34
Removed highly correlated: 8
Final selected features: 26
Processing file: part-00005-363d1ba3-8ab5-4f96-bc25-4d5

In [1]:
final_voted_features=['urg_count', 'Min', 'syn_flag_number', 'Duration', 'flow_duration', 'Protocol Type', 'rst_count', 'Variance', 'Header_Length', 'HTTP', 'UDP', 'TCP', 'Max', 'syn_count', 'Rate', 'Tot size', 'ICMP', 'psh_flag_number', 'Covariance', 'AVG', 'fin_flag_number', 'ack_flag_number', 'Tot sum', 'HTTPS', 'IAT', 'fin_count']

In [3]:
!pip install seaborn matplotlib

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/d7/68/0d03098b3feb786cbd494df0aac15b571effda7f7cbdec267e8a8d398c16/matplotlib-3.10.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/a8/7e/cd93cab453720a5d6cb75588cc17dcdc08fc3484b9de98b885924ff61900/contourpy-1.3.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/13/4c/de2612ea2216eb45cfc8eb91a8501615dd87716feaf5f8fb65cbca


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
path

'./CICIoT2023/'

In [ ]:
import os
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

selected_features = ['urg_count', 'Min', 'syn_flag_number', 'Duration', 'flow_duration',
                     'Protocol Type', 'rst_count', 'Variance', 'Header_Length', 'HTTP', 'UDP',
                     'TCP', 'Max', 'syn_count', 'Rate', 'Tot size', 'ICMP', 'psh_flag_number',
                     'Covariance', 'AVG', 'fin_flag_number', 'ack_flag_number', 'Tot sum',
                     'HTTPS', 'IAT', 'fin_count']

path_dataset = path
csv_files = [f for f in os.listdir(path_dataset) if f.endswith('.csv')]
total_files = len(csv_files)

try:
    from xgboost import XGBClassifier
    model_class = XGBClassifier
    model_params = {
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'n_estimators': 100,
        'random_state': 42,
        'verbosity': 1,
        'use_label_encoder': False,
        'eval_metric': 'mlogloss'
    }
except ImportError:
    from sklearn.ensemble import RandomForestClassifier
    model_class = RandomForestClassifier
    model_params = {'n_estimators': 100, 'random_state': 42, 'n_jobs': -1}
    print("⚠️ XGBoost not installed. Using RandomForest instead.")

df_list = []
for idx, file in enumerate(csv_files, 1):
    file_path = os.path.join(path_dataset, file)
    print(f"[{idx}/{total_files}] Processing file: {file}")

    try:
        df = pd.read_csv(file_path)
        if set(selected_features + ['label']).issubset(df.columns):
            df_list.append(df[selected_features + ['label']])
        else:
            print("  - Skipped: Missing required features.")
    except Exception as e:
        print(f"  - Error reading file: {e}")

if not df_list:
    raise ValueError("❌ No valid CSV files found with the required features.")

full_df = pd.concat(df_list, ignore_index=True)
print(f"\nCombined dataset shape: {full_df.shape}")

full_df = full_df.sample(frac=1, random_state=42).reset_index(drop=True)

le = LabelEncoder()
full_df['label_encoded'] = le.fit_transform(full_df['label'])
X = full_df[selected_features]
y = full_df['label_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nStarting model training...\n")
start_time = time.time()
model = model_class(**model_params)
model.fit(X_train, y_train)
train_time = time.time() - start_time

print(f"Training completed in {train_time/60:.2f} minutes")

y_pred = model.predict(X_test)

print("\nEvaluation Results:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

[1/169] Processing file: part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[2/169] Processing file: part-00001-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[3/169] Processing file: part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[4/169] Processing file: part-00003-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[5/169] Processing file: part-00004-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[6/169] Processing file: part-00005-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[7/169] Processing file: part-00006-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[8/169] Processing file: part-00007-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[9/169] Processing file: part-00008-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[10/169] Processing file: part-00009-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[11/169] Processing file: part-00010-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[12/169] Processing file: part-00011-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv
[13/169] Proc

c:\Users\Administrator\Desktop\nis\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:08:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Administrator\Desktop\nis\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:08:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [11]:
import os
df_sets = [k for k in os.listdir(path) if k.endswith('.csv')]
df_sets.sort()
training_sets = df_sets[:int(len(df_sets)*.67)]
test_sets = df_sets[int(len(df_sets)*.67):]

X_columns = ['urg_count', 'Min', 'syn_flag_number', 'Duration', 'flow_duration',
                     'Protocol Type', 'rst_count', 'Variance', 'Header_Length', 'HTTP', 'UDP',
                     'TCP', 'Max', 'syn_count', 'Rate', 'Tot size', 'ICMP', 'psh_flag_number',
                     'Covariance', 'AVG', 'fin_flag_number', 'ack_flag_number', 'Tot sum',
                     'HTTPS', 'IAT', 'fin_count']

y_column = 'label'

In [3]:
!pip install xgboost


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from xgboost import XGBClassifier
params = {
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'n_estimators': 100,
    'random_state': 42,
    'eval_metric': 'mlogloss'
    
}
ML_models = [
        RandomForestClassifier(n_estimators=50,random_state=42,class_weight='balanced'),
        XGBClassifier(n_estimators=50)
        ]

ML_names = [
        "Random Forest",
        "XG BOOST"
]

In [13]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer(output_distribution='uniform')
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_class_weight

In [15]:
for train_set in tqdm(training_sets):
        d = pd.read_csv(path + train_set)
        d[X_columns] = scaler.fit_transform(d[X_columns])
        X_train = d[X_columns].values
        y_train = d[y_column].values
        from sklearn.preprocessing import LabelEncoder
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        for model in ML_models:
        # Fit the model with class weights
            model.fit(X_train, y_train_encoded)
        del d   

100%|██████████| 113/113 [17:23<00:00,  9.23s/it]


In [17]:
import pickle

with open ('label_enc.pkl', 'wb') as f:
    pickle.dump(label_encoder,f)

In [18]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import matplotlib.pyplot as plt
y_test = []
y_pred_proba=[]
preds = {i:[] for i in range(len(ML_models))}
for test_set in tqdm(test_sets):
    d_test = pd.read_csv(path + test_set)
    d_test[X_columns] = scaler.fit_transform(d_test[X_columns])
    y_test += list(d_test[y_column].values)


    for i in range(len(ML_models)):
        model = ML_models[i]
        y_pred = list(model.predict(d_test[X_columns]))
        y_pred_proba += list(model.predict_proba(d_test[X_columns])[::,1])
        preds[i] = preds[i] + y_pred
        del y_pred
    del d_test

100%|██████████| 56/56 [01:44<00:00,  1.86s/it]


In [ ]:
with open('label_enc.pkl','rb') as f:
    le = pickle.load(f)



from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report
for k,v in preds.items():
    y_pred = v
    y_pred = le.inverse_transform(y_pred)
    print(confusion_matrix(y_test, y_pred))
    print("transfered")
    print("XGBoost")
    # print(f"{ML_names[k]}")
    print(classification_report(y_test,y_pred))
    print('accuracy_score: ', accuracy_score(y_pred, y_test))
    print('recall_score: ', recall_score(y_pred, y_test, average='macro'))
    print('precision_score: ', precision_score(y_pred, y_test, average='macro'))
    print('f1_score: ', f1_score(y_pred, y_test, average='macro'))



[[    27    157      3 ...      0      0      8]
 [     2 357603     46 ...      0      0     17]
 [     2    487     62 ...      0      0      5]
 ...
 [     0     63      2 ...      0      0      1]
 [     0    274      0 ...      0   5745      7]
 [    13    217     11 ...      0      0      6]]
transfered
XGBoost
                         precision    recall  f1-score   support

       Backdoor_Malware       0.16      0.02      0.04      1131
          BenignTraffic       0.85      0.92      0.89    387031
       BrowserHijacking       0.16      0.03      0.05      2041
       CommandInjection       0.57      0.13      0.21      1863
 DDoS-ACK_Fragmentation       1.00      0.98      0.99    100231
        DDoS-HTTP_Flood       0.90      0.82      0.86     10131
        DDoS-ICMP_Flood       1.00      1.00      1.00   2537902
DDoS-ICMP_Fragmentation       0.99      0.99      0.99    160098
      DDoS-PSHACK_Flood       1.00      1.00      1.00   1445379
       DDoS-RSTFINFlood       

In [20]:
import pickle
import os

xgb_model = ML_models[0]

mpath = './xgb_opt.pkl'
with open(mpath, 'wb') as f:
    pickle.dump(xgb_model,f)

In [22]:
path

'./CICIoT2023/'

In [23]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import joblib

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

final_features = ['urg_count', 'Min', 'syn_flag_number', 'Duration', 'flow_duration',
                     'Protocol Type', 'rst_count', 'Variance', 'Header_Length', 'HTTP', 'UDP',
                     'TCP', 'Max', 'syn_count', 'Rate', 'Tot size', 'ICMP', 'psh_flag_number',
                     'Covariance', 'AVG', 'fin_flag_number', 'ack_flag_number', 'Tot sum',
                     'HTTPS', 'IAT', 'fin_count']

path_dataset = path
csv_files = [f for f in os.listdir(path_dataset) if f.endswith('.csv')]

normal_df_list = []
for file in tqdm(csv_files):
    try:
        df = pd.read_csv(os.path.join(path_dataset, file))
        if set(final_features + ['label']).issubset(df.columns):
            normal_df = df[df['label'] == 'BenignTraffic']
            if not normal_df.empty:
                normal_df_list.append(normal_df[final_features])
    except Exception as e:
        print(f"Error in {file}: {e}")

if not normal_df_list:
    raise ValueError("No 'BenignTraffic' traffic found.")

normal_data = pd.concat(normal_df_list, ignore_index=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(normal_data)
joblib.dump(scaler, "stage2_qae_scaler.pkl")

X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)

class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, input_dim)
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

input_dim = X_tensor.shape[1]
model = Autoencoder(input_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 50
batch_size = 128
dataloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(X_tensor, X_tensor),
    batch_size=batch_size,
    shuffle=True
)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for x_batch, y_batch in dataloader:
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} Loss: {epoch_loss / len(dataloader):.6f}")

torch.save(model.state_dict(), "stage2_qae_model.pth")
print("Model saved as stage2_qae_model.pth")

Using device: cuda


100%|██████████| 169/169 [01:23<00:00,  2.02it/s]


Epoch 1/50 Loss: 0.255868
Epoch 2/50 Loss: 0.188395
Epoch 3/50 Loss: 0.151688
Epoch 4/50 Loss: 0.145910
Epoch 5/50 Loss: 0.143957
Epoch 6/50 Loss: 0.143599
Epoch 7/50 Loss: 0.142376
Epoch 8/50 Loss: 0.141594
Epoch 9/50 Loss: 0.141402
Epoch 10/50 Loss: 0.140786
Epoch 11/50 Loss: 0.140441
Epoch 12/50 Loss: 0.140210
Epoch 13/50 Loss: 0.139989
Epoch 14/50 Loss: 0.139300
Epoch 15/50 Loss: 0.138518
Epoch 16/50 Loss: 0.138160
Epoch 17/50 Loss: 0.138021
Epoch 18/50 Loss: 0.137663
Epoch 19/50 Loss: 0.137693
Epoch 20/50 Loss: 0.137636
Epoch 21/50 Loss: 0.137528
Epoch 22/50 Loss: 0.137372
Epoch 23/50 Loss: 0.137383
Epoch 24/50 Loss: 0.137231
Epoch 25/50 Loss: 0.137434
Epoch 26/50 Loss: 0.137360
Epoch 27/50 Loss: 0.137208
Epoch 28/50 Loss: 0.137310
Epoch 29/50 Loss: 0.137090
Epoch 30/50 Loss: 0.137039
Epoch 31/50 Loss: 0.136947
Epoch 32/50 Loss: 0.136957
Epoch 33/50 Loss: 0.136989
Epoch 34/50 Loss: 0.137016
Epoch 35/50 Loss: 0.136832
Epoch 36/50 Loss: 0.136914
Epoch 37/50 Loss: 0.136880
Epoch 38/5

In [ ]:
import torch
import torch.nn as nn
import joblib
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the same architecture
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, input_dim)
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

scaler = joblib.load("stage2_qae_scaler.pkl")

input_dim = 26  # Must match training
model = Autoencoder(input_dim).to(device)
model.load_state_dict(torch.load("stage2_qae_model.pth", map_location=device))
model.eval()

raw_sample = np.array([[0.0, 572.4, 0.0, 64.0, 29.18872833251953, 6.0, 0.0, 0.9, 1295005.2, 0.0, 0.0, 1.0, 2962.0, 1.8, 79.6554195521552, 1523.0, 0.0, 0.0, 615312.3146456915, 1744.0607142857143, 0.0, 1.0, 8688.6, 1.0, 0.0006201982498168, 0.0]])  # shape (1, 26)


scaled_sample = scaler.transform(raw_sample)
tensor_input = torch.tensor(scaled_sample, dtype=torch.float32).to(device)

with torch.no_grad():
    reconstructed = model(tensor_input)

mse = nn.MSELoss()(reconstructed, tensor_input).item()
print("Reconstruction Error:", mse)

Reconstruction Error: 0.046370938420295715


In [ ]:
import onnxmltools
from onnxmltools.convert.common.data_types import FloatTensorType
from xgboost import XGBClassifier
import numpy as np

initial_type = [("input", FloatTensorType([None, X_train.shape[1]]))]

onnx_model = onnxmltools.convert_xgboost(xgb_model, name="xgboost_model", initial_types=initial_type)

onnx_filename = "xgboost_model.onnx"
with open(onnx_filename, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"Model saved as {onnx_filename}")
